# 使用循环来实现卷积层

2019 年 5 月 19 日

在 [使用 Numpy 中的 as_strided 与 tensordot 实现卷积层](./使用%20Numpy%20中的%20as_strided%20与%20tensordot%20实现卷积层.ipynb) (即当前目录下的 `使用 Numpy 中的 as_strided 与 tensordot 实现卷积层.ipynb` 文件) 这篇文章中, 当经过 padding 后的输入图像的长度无法被 stride 整除的时候, 代码就无法处理. 为了解决这个问题, 我尝试使用循环来实现卷积层, 之后再想办法优化.

## 辅助函数

这些函数在前一篇文章中已经说明了.

In [5]:
import numpy as np

def make_padding(input, padding=(0, 0)):
    if padding == (0, 0):
        return input
    B, C, H, W = input.shape
    pad = np.zeros((B, C, H + 2 * padding[0], W + 2 * padding[1]))
    pad[..., padding[0]:-padding[0], padding[1]:-padding[1]] = input
    return pad

def make_dilation(input, dilation=(1, 1)):
    if dilation == (1, 1):
        return input
    
    B, C, H, W = input.shape
    p, q = dilation
    oh, ow = p * (H - 1) + 1, q * (W - 1) + 1
    pad = np.zeros((B, C, oh, ow))
    pad[..., ::p, ::q] = input
    return pad

def unwrap_padding(input, padding=(0, 0)):
    if padding == (0, 0):
        return input
    p, q = padding
    return input[..., p:-p, q:-q]


def rotate_kernel(kernel):
    return kernel[..., ::-1, ::-1]

## 前向传播的实现

下面函数中, 首先求出输出特征的大小, 然后以输出特征为视角 (即针对输出特征中的每一个点 `out[n, c, i, j]`), 在输入特征中查找贡献于这个点的区域, 当考虑 stride 的时候, 这个区域为 `input[n, :, i * s : i * s + kh, j * s : j * s + kw]`, (注意 `i` 从 0 开始), 对应的 kernel 为 `kernel[:, c, ...]`, 两者相乘之后, 要对所有位置的元素进行求和.

In [12]:
def conv(input, kernel, p, s, d):
    input = make_padding(input, (p, p))
    
    N, C, H, W = input.shape
    iC, oC, kh, kw = kernel.shape
    assert C == iC, 'channels not aligned'
    
    oh, ow = (H - (d * (kh - 1) + 1)) // s + 1, (W - (d * (kw - 1) + 1)) // s + 1
    out = np.zeros((N, oC, oh, ow))
    
    for n in range(N):
        for c in range(oC):
            for i in range(oh):
                for j in range(ow):
                    out[n, c, i, j] = np.sum(input[n, :, i * s : i * s + kh, j * s : j * s + kw] * \
                                             kernel[:, c, ...])
                    
    return out

## 反向传播的实现

分为两个部分, 分别是误差传播以及梯度更新. 首先来看误差传播.

### 误差传播

首先, `input_grad` 的大小肯定和 `input` 的大小一样. 在下面的循环中, (如果有个图来表达我的思想就好了), 想象一张 feature map 上和 kernel 同样大的区域, 在这个区域中的每个像素都引出一条线指向输出 feature map 中的某个点. 好吧, 就如下图所示:

 <img src="./figures/convolution.png" width = "300" height = "200" alt="图片名称" align=center />

如果从这种视角出发, 那么对于输出 feature map 中的点 (表示 `eta` 中的某点), 它只要和连接线上的权重 (假设每条线上有权重) 相乘, 就可以得到对应的输入 feature map 上的点的梯度. 从这个角度出发, 就无需将 kernel 进行翻转了, (将 kernel 翻转在前一篇博客 [使用 Numpy 中的 as_strided 与 tensordot 实现卷积层](./使用%20Numpy%20中的%20as_strided%20与%20tensordot%20实现卷积层.ipynb) 中有讨论, 那个时候是先对 `eta` 进行 padding, 然后再将 `eta` 和 kernel 进行卷积, 这个时候, kernel 需要进行翻转), 因为这里是对 `input_grad` 先进行 padding, 所以说, 还是一个计算时的视角问题. 这种方法大大简化了问题, 那么对于输入 feature map 中的区域 `input_grad[n, c, i * s : i * s + kh, j * s : j * s + kw]` (考虑了 stride), 它对每个输出 feature map 中位于 `(i, j)` 处的点都有贡献, 因此需要将 `eta[n, :, i, j]` 与 `kernel[c, :, ...]` 进行相乘, 注意相乘的结果为 `shape=(oC, kh, kw)` 大小的矩阵, 因此要对 `axis=0` 即 `oC` 这个轴进行求和. 最后由于 `input_grad` 是经过 padding 的, 此时将 padding 使用 `unwrap_padding` 去掉即可.

### 梯度更新

梯度更新, 方法是输入 feature map 和经过 dilation 的 eta 进行卷积. 在 [Part 2: Backpropagation for Convolution with Strides](https://medium.com/@mayank.utexas/backpropagation-for-convolution-with-strides-fb2f2efc4faa)  这篇博客中, 有漂亮的动图来分析这个问题. 其实自己画图分析一下也可以发现在 stride 的情况下, 哪些像素给输出 feature map 贡献了, 同时 kernel 在哪些位置给输出 feature map 贡献了. 答案就是将经过 dilation 的 eta 贴在经过 padding 后的输入 feature map 上.

 <img src="./figures/kernel_grad.png" width = "300" height = "200" alt="图片名称" align=center />

不用尝试看懂这张图, 自己去画画图就很容易明白. 总结起来就是, 如果 kernel 在 input 上移动存在 stride, 那么 kernel 中的每个 $w_i$ 的移动轨迹, 和使用经过 dilation 后(dilation 的程度等于 stride)的输出 feature map 在 input 上的移动轨迹相同.

那么对于 kernel 中的每个点 `kernel_grad[ic, oc, i, j]`, 结果就是输入 (经过 padding 的 input) 和 `eta` (经过了 dilation) 进行卷积得到.

In [11]:
def backward_conv(input, kernel, eta, p, s, d):
    N, C, h, w = input.shape
    iC, oC, kh, kw = kernel.shape
    oh, ow = eta.shape[-2:]
    input_grad = np.zeros_like(input)
    kernel_grad = np.zeros_like(kernel)
    
    ## 误差传播
    ## 由于我是直接对矩阵相乘, 所以这里不需要将 kernel 翻转.
    input_grad = make_padding(input_grad, (p, p))
    for n in range(N):
        for c in range(C):
            for i in range(oh):
                for j in range(ow):
                    input_grad[n, c, i * s : i * s + kh, j * s : j * s + kw] += np.sum(kernel[c, :, ...] * \
                                                                            eta[n, :, i, j], axis=0)
    input_grad = unwrap_padding(input_grad, (p, p))

    ## 梯度更新
    input = make_padding(input, (p, p))
    eta = make_dilation(eta, (s, s))
    dh, dw = eta.shape[-2:]
    for ic in range(iC):
        for oc in range(oC):
            for i in range(kh):
                for j in range(kw):
                    kernel_grad[ic, oc, i, j] = np.sum(input[:, ic, i : i + dh, j : j + dw] * eta[:, oc, ...])
    return input_grad, kernel_grad

## 例子验证

In [13]:
p = 0
s = 1
d = 1

h = w = 5
kh = kw = 2
N = 1
C = 1
oC = 1

input = np.arange(N * C * h * w, dtype=np.float64).reshape(N, C, h, w)
# kernel = np.arange(kh * kw, dtype=np.float64).reshape(1, 1, kh, kw)
kernel = np.ones((C, oC, kh, kw))
out_conv = conv(input, kernel, p, s, d)
eta = np.ones_like(out_conv)
input_grad, kernel_grad = backward_conv(input, kernel, eta, p, s, d)
print(input_grad)
print(kernel_grad)

[[[[1. 2. 2. 2. 1.]
   [2. 4. 4. 4. 2.]
   [2. 4. 4. 4. 2.]
   [2. 4. 4. 4. 2.]
   [1. 2. 2. 2. 1.]]]]
[[[[144. 160.]
   [224. 240.]]]]


## PyTorch 的结果


In [15]:
import torch
import torch.nn as nn
from torch.autograd import Variable

input = Variable(torch.arange(N * C * h * w).view(N, C, h, w).float(), requires_grad=True)
net = nn.Conv2d(C, oC, (kh, kw), padding=p, stride=s, bias=False)
shape = net.weight.data.size()
# net.weight.data.copy_(torch.arange(kh * kw).view(shape))
net.weight.data.copy_(torch.ones_like(net.weight.data))
output = net(input)
# print(output)
y = output.sum()
# print(y)
y.backward()
print(input.grad)
print(net.weight.grad)

tensor([[[[1., 2., 2., 2., 1.],
          [2., 4., 4., 4., 2.],
          [2., 4., 4., 4., 2.],
          [2., 4., 4., 4., 2.],
          [1., 2., 2., 2., 1.]]]])
tensor([[[[144., 160.],
          [224., 240.]]]])
